###### Import Dependencies & Chrome Driver

In [1]:
# import dependencies
from bs4 import BeautifulSoup as soup
import pandas as pd
from pprint import pprint
from splinter import Browser
import time

In [2]:
# choose path to driver
executable_path = {'executable_path': 'chromedriver'} # probably repoint this to repo for HW
browser = Browser(
    'chrome', 
    **executable_path, 
    headless = False
)

# delay to allow time for page to load
time.sleep(5)

###### Scraping Web Text

In [3]:
# navigate browser to url
browser.visit('https://mars.nasa.gov/news/')

# create beautifulsoup object
article_soup = soup(browser.html, 'html.parser')

# delay to allow time for page to load
time.sleep(2)

In [4]:
# narrow HTML elements by 'article'
article_result = article_soup.find('article')

# retrieve most recent article date, title, paragraph
news_date = article_result.find('div', class_ = 'list_date').text.strip()
news_title = article_result.find('div', class_ = 'content_title').text.strip()
news_p = article_result.find('div', class_ = 'article_teaser_body').text.strip()

###### Scraping Web Image

In [5]:
# navigate browser to url, click full-res image
browser.visit('https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html')
browser.links.find_by_partial_text('FULL IMAGE').click()

# create beautifulsoup object
image_soup = soup(browser.html, 'html.parser')

# delay to allow time for page to load
time.sleep(5)

In [6]:
# extract image source url
image_result = image_soup.find(class_ = 'fancybox-image')['src']

In [7]:
# append urls
featured_image_url = 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/' + image_result

###### Converting HTML table to DataFrame, convert back to HTML

In [8]:
# navigate browser to url
browser.visit('https://space-facts.com/mars/')

# create beautifulsoup object
fact_soup = soup(browser.html, 'html.parser')

# delay to allow time for page to load
time.sleep(5)

In [9]:
# convert html table to dataframe
fact_df = pd.read_html('https://space-facts.com/mars/')[0]

In [10]:
# convert dataframe to html
fact_html = fact_df.to_html(index = False, header = False)

###### Mars Hemispheres

In [11]:
# navigate browser to url
browser.visit('https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars')

# create beautifulsoup object
hemi_soup = soup(browser.html, 'html.parser')

# delay to allow time for page to load
time.sleep(5)

In [12]:
# narrow HTML elements by 'xxx'
hemisphere_result = hemi_soup.find()

# grab hemisphere HTML elements
hemisphere_content = hemi_soup.find_all('div', class_ = 'item')

In [13]:
# create list for hemisphere info
hemisphere_images = []

# retrieve hemisphere info
for hemisphere in hemisphere_content:
    hemi_url = {}
    title = hemisphere.find('div', class_='description').h3.text
    hemi_url['title'] = title
    
    # delay to allow time for page to load
    time.sleep(1)
    browser.find_by_text(title).click()
    hemi_soup = soup(browser.html, 'html.parser')
    
    # append img urls for full-res imgs
    download = hemi_soup.find('div', class_ = "downloads")
    download_link = download.find('a')
    if download_link.text == 'Sample':
        img_url = download_link['href']
        hemi_url['img_url'] = img_url
    
    # append urls to hemisphere dictionary
    hemisphere_images.append(hemi_url)
    time.sleep(1)
    browser.back()

In [14]:
# close browser
browser.quit()